In [1]:
import plotly, pandas, os
# check environment
print(os.environ['CONDA_DEFAULT_ENV'])
print(pandas.__version__, plotly.__version__)

stemcellhack
0.25.0 4.0.0


In [212]:
# read into data frame
df = pandas.read_csv("imac_atlas_expression_v7.1.tsv", sep='\t', index_col=0)
df.head()

,1674120023_B;1000,1674120023_F;1000,1674120053_B;1000,1674120053_C;1000,1674120078_A;1000,1674120078_F;1000,1674120079_D;1000,1674120079_E;1000,1674120084_B;1000,1674120084_D;1000,...,L;7002,M;7002,N;7002,O;7002,P;7002,Q;7002,R;7002,S;7002,T;7002,U;7002
ENSG00000000003,0.023640,0.090933,0.114154,0.122709,0.094218,0.066109,0.010350,0.013863,0.073289,0.066835,...,0.307822,0.152536,0.212954,0.107241,0.339368,0.338222,0.368622,0.267415,0.258097,0.039108
ENSG00000000005,0.391079,0.249771,0.270929,0.233692,0.305683,0.296861,0.168996,0.316109,0.336350,0.330698,...,0.106783,0.119768,0.067369,0.135121,0.144363,0.125038,0.134586,0.130614,0.141002,0.140009
ENSG00000000419,0.890315,0.862511,0.901008,0.920639,0.901543,0.880690,0.880690,0.911778,0.912618,0.859762,...,0.932860,0.953483,0.938130,0.937825,0.959212,0.949511,0.947984,0.960281,0.943782,0.950351
ENSG00000000938,0.991292,0.993737,0.993355,0.992285,0.993126,0.993737,0.994119,0.992209,0.991674,0.994348,...,0.589902,0.976551,0.987091,0.987015,0.937748,0.938894,0.957149,0.991903,0.991980,0.991369
ENSG00000000971,0.641537,0.574473,0.585625,0.671746,0.620035,0.595134,0.561564,0.628246,0.627330,0.573060,...,0.099297,0.254277,0.137565,0.220975,0.104644,0.218607,0.244424,0.200122,0.255194,0.220593


In [213]:
input = [[1,2,4], [2,3,2], [3,5,3], [5,7,3], [4,6,1], [4,9,9], [9,3,8], [6,8,8], [2,5,6], [7,8,0], [8,5,3], [6,2,1], [9,10,3]]
# convert list input into data frame
input = pandas.DataFrame(input, columns=["x", "y", "z"])

import pandas, plotly.express as px
def scatterGraph(input, dim):
    """
    Args:
        input (DataFrame): points to feed in
        dim (int): number of dimensions to graph in
    Returns:
        nothing, plots nice looking scatterplot in 2D or 3D
    """
    if dim == 2:
        input.columns = ['x', 'y']
        fig = px.scatter(input, x="x", y="y")
    elif dim == 3:
        input.columns = ['x', 'y', 'z']
        fig = px.scatter_3d(input, x="x", y="y", z="z")
    fig.show()

scatterGraph(input, 3)

In [214]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def pcaTransform(input, retain):
    """
    Args:
        input (DataFrame): input points to be PCA transformed
            columns are dimensions to be reduced
        retain (int): number of dimensions to reduce to
    Returns:
        DataFrame of PCA transformed points
    """
    # store columns of input data frame
    cols = [i for i in input.columns]
    # Z score normalize so no one feature stands out over the others
    input = StandardScaler().fit_transform(input)
    input = pandas.DataFrame(input, columns = cols)
    # create pca object to analyze n dimensions (number of columns)
    pca = PCA(n_components=len(input.columns))
    
    # feed input in to be analyzed
    pca.fit(input)
    
    # transform points according to PCA to minimize variances
    points = pca.transform(input)
    # add back column labels to points
    points = pandas.DataFrame(points, columns=cols)
    
    # proportion of variance explained by each dimension
    var = pca.explained_variance_ / sum(pca.explained_variance_)
    # make sure matrix is horizontal 1x3
    var = pandas.DataFrame(var)
    # label this column variance explained
    var.columns = ['variance explained']
    # add column labels on top of 1x3 matrix
    var.insert(1, 'col variable', cols)

    # reduce dimension to number specified by retain
    pcaReduce = PCA(n_components=retain)
    points = pandas.DataFrame(pcaReduce.fit_transform(points))
    
    # tidy table has all graph variables as columns, rows are just IDs
    fig = px.bar(var, x='col variable', y='variance explained')
    fig.show()

    return points

retain = 50
points = pcaTransform(df, retain)


In [221]:
points = points.transpose()

In [223]:
pcaTransform(points, 200)

ValueError: n_components=200 must be between 0 and min(n_samples, n_features)=50 with svd_solver='full'